In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from datetime import timedelta, date

import matplotlib.dates as mdates
from scipy import stats
import datetime as dt
import psycopg2

connection = psycopg2.connect(user = "postgres", password = "admin", host = "127.0.0.1",port = "5432", database = "postgres")

def close_conn():
    if(connection):
        cursor.close()
        connection.close()


In [2]:
def get_data(stock):
    data = pd.read_sql_query("SELECT * FROM finance.stock_price where stock = '"+stock+"' and date>'2018-01-01' order by date asc",con=connection)
    return data

def get_stocks():
    data = pd.read_sql_query("SELECT a.stock, cap, vol, beta from "+
                            "( "+
                            "SELECT avg(marketcap) as cap, avg(beta) as beta, stock "+
                            "  FROM finance.stock_valuation_stats  where date > '2019-01-01' group by stock "+
                            ") a, " +
                            "( "+
                            "SELECT avg(vol) as vol, stock "+
                            "FROM finance.stock_price where date > '2019-01-01' group by stock "+
                            ") b "+
                            "  WHERE a.stock = b.stock "+
                            "    and cap is not null and beta >1 "+
                            "  order by cap desc, vol desc, beta desc ",con=connection)
    return data

# 1. Proccess Data

In [3]:
class Strategy:
    def get_action(self, data, i, shares_can_buy, shares_can_sell, cash):
        pass
    
    def set_stoploss(self, data, i, atr_period = 14, atr_multiplier = 3):
        data.loc[i+1:,("stoploss")] = ATR_STOPLOSS(close = np.array(data["close"][i-atr_period:]), 
                                                         high = np.array(data["high"][i-atr_period:]),
                                                         low = np.array(data["low"][i-atr_period:]), 
                                     times=atr_multiplier, period=atr_period)[0][atr_period+1:]
        return data


class StrategyRandom(Strategy):
    def get_action(self, data, i, shares_can_buy, shares_can_sell):
        import random
        action ="HOLD", 0 , data
        rand = random.uniform(0, 1)
        if(rand < 0.6
          and i>14
          ):
            data = self.set_stoploss(data, i , 14, 3)
            action = "BUY", data.close[i], data
        #f(rand > 0.7):
        #    action = "SELL", data.close[i]

        if(shares_can_sell>0):
            if(i>0 
               and data["stoploss"][i]>data.low[i]
              ):
                action = "STOPPED", data["stoploss"][i], data
        
        return action
    
    

def println(str):
    show_each_tries= True
    if(show_each_tries):
        print(str)

def back_test(data, cash = 100000,transaction_fee = 0.01, assets={}, strategy=StrategyRandom(), min_transaction = 1000):
    longs = {}
    shorts = {}
    init_cash = cash
    for i,row in data.iterrows():
        shares_can_buy = int(cash/(row.close*(1+transaction_fee)))
        shares_can_sell=0
        if row.stock in assets:
            shares_can_sell = assets[row.stock]
        #Decision
        recommended_shares = -1
        shares_held = 0
        if(i>0):
            current_assets = cash
            if(data.iloc[i].stock in assets):
                shares_held = assets[data.iloc[i-1].stock]
                current_assets = shares_held*data.iloc[i-1].close + cash
            risk_percent = 0.01
            risk_multiplier = 3
            risk_atr = data["atr_risk"][i-1]
            last_close = data["close"][i-1]
            import math
            if(math.isnan(risk_atr)==False):
                risk_atr =  risk_atr * risk_multiplier
                risk_amt = current_assets * risk_percent
                recommended_shares = int(risk_amt/risk_atr)
                if(shares_can_buy>0):
                    if(shares_held+shares_can_buy>recommended_shares):
                        shares_can_buy = int(recommended_shares-shares_held)
            else:
                continue
        if(recommended_shares<=0 or recommended_shares*row.close*(1+transaction_fee)<min_transaction):
            continue
        action , price_point, data = strategy.get_action(data, i, shares_can_buy, shares_can_sell)
        #Execution
        if(action=="BUY"):
            if(shares_can_buy>0):
                cost = shares_can_buy*price_point*(1+transaction_fee)
                cash_left = cash-cost
                if row.stock not in assets:
                    assets[row.stock]=0
                assets[row.stock] = assets[row.stock] + shares_can_buy
                cash = cash_left
                longs[row.date] = price_point
                #print("BUY : +" +str(shares_can_buy) +" @"+str(price_point))
        if(action=="SELL"):
            if(shares_can_sell>0):
                asset = shares_can_sell * price_point
                cash = cash + asset*(1-transaction_fee)
                assets.clear()
                shorts[row.date] = price_point
                #print("SELL : -" +str(shares_can_sell) +" @"+str(price_point))
        if(action=="STOPPED"):
            if(shares_can_sell>0):
                asset = shares_can_sell * price_point
                cash = cash + asset*(1-transaction_fee)
                assets.clear()
                shorts[row.date] = price_point
                #print("STOPPED : -" +str(shares_can_sell) +" @"+str(price_point))
    final_cash = cash
    if(data.iloc[-1].stock in assets):
        final_cash = assets[data.iloc[-1].stock]*data.iloc[-1].close + cash
    assets.clear()
    println("asset: "+str(final_cash))
    growth =((final_cash-init_cash)/init_cash)*100
    println("profit: "+str(round(growth,2))+"%")
    return round(growth,2), longs, shorts


In [264]:
def ATR_STOPLOSS(close, high, low, times=3, period=14):
    from talib import ATR
    atr= ATR(high,low, close ,timeperiod=period)
    first = 1
    import math
    atr_trailing_stoploss = np.full(len(close)+1, np.nan)
    for i in range(0,len(close)):
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]>atr_trailing_stoploss[i] and close[i]<atr_trailing_stoploss[i]):
                loss = atr[i]*times
                atr_trailing_stoploss[i+1]=high[i]+loss  
                continue
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]<atr_trailing_stoploss[i] and close[i]>atr_trailing_stoploss[i]):
                loss = atr[i]*times
                atr_trailing_stoploss[i+1]=low[i]-loss  
                continue
        if(math.isnan(atr[i])==False):
            loss = atr[i]*times
            atr_trailing_stoploss[i+1]=low[i]-loss
            if(close[i]<atr_trailing_stoploss[i]):
                if(atr_trailing_stoploss[i]>high[i]+loss):
                    atr_trailing_stoploss[i+1]=high[i]+loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
            else:
                if(atr_trailing_stoploss[i]<low[i]-loss):
                    atr_trailing_stoploss[i+1]=low[i]-loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
    return atr_trailing_stoploss[:-1], atr_trailing_stoploss[-1:]

def Normalise(listss):
    min_i = min(listss)
    max_i = max(listss)
    if(min_i==max_i):
        return listss
    return [(x-min_i)/(max_i-min_i) for x in listss]
def SLOPE(data):
    #data = np.array(Normalise(data))
    #data = data * (len(data)-1)
    from talib import LINEARREG_ANGLE
    return list(LINEARREG_ANGLE(data,timeperiod = len(data)-1))[-1]
def PRICE_SLOPE(data, period = 14):
    df = data.tail(period)
    first = df.index.start
    last = df.index.stop
    x = df.index.values.reshape(-1, 1) 
    y= df.close.values.reshape(-1, 1) 
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(x,y)
    first_predicted = reg.predict(np.array([[first]]))[0][0]
    last_predicted = reg.predict(np.array([[last-1]]))[0][0]
    gradient =(last_predicted-first_predicted)/(last-first)
    return gradient


def HEIKIN_ASHI(open_price, high_price, low_price, close_price):
    ha_close = (open_price+ high_price+ low_price+ close_price) / 4
    ha_open = np.full(len(open_price), np.nan)
    ha_high = np.full(len(open_price), np.nan)
    ha_low = np.full(len(open_price), np.nan)
    for i in range(len(open_price)):
        if i == 0:
            ha_open[0]= open_price[0]
        else:
            ha_open[i] = (ha_open[i-1] + ha_close[i-1]) / 2
        ha_high[i] = np.array([high_price[i], ha_open[i], ha_close[i]]).max()
        ha_low[i] = np.array([low_price[i], ha_open[i], ha_close[i]]).min()
    return ha_open, ha_high, ha_low, ha_close

    '''
    if(i>60):
            last_60days_trend_slope = PRICE_SLOPE(data[0:i], period = 60)
            last_60days_max_high = data[0:i].tail(60).high.max()
            last_60days_min_low = data[0:i].tail(60).low.min()
    '''

    '''
        if(i>30):
            last_30days_trend_slope = PRICE_SLOPE(data[0:i], period = 30)
            last_30days_max_high = data[0:i].tail(30).high.max()
            last_30days_min_low = data[0:i].tail(30).low.min()
    '''

    '''
        if(i>14):
            last_14days_trend_slope = PRICE_SLOPE(data[0:i], period = 14)
            last_14days_max_high = data[0:i].tail(14).high.max()
            last_14days_min_low = data[0:i].tail(14).low.min()
            price_perct_14days = (data["close"][i]-last_14days_min_low)/(last_14days_max_high-last_14days_min_low)
    '''

    '''
        max_rsi = data["rsi"][0:i].max()
        min_rsi = data["rsi"][0:i].min()
        rsi_threshold = (max_rsi-min_rsi)*0.02
    '''

def LINEAR_REGRESSION_BOUND(data_close, data_high, data_low, period = 14, skip = 0):
    linreg_upper = np.full(len(data_close), np.nan)
    linreg_lower = np.full(len(data_close), np.nan)
    for i in range(len(data_close)):
        if(i>(period-1+skip)):
            close = np.array(data_close[i-(period)-skip:i-skip])
            high = np.array(data_high[i-(period)-skip:i-skip])
            low = np.array(data_low[i-(period)-skip:i-skip])
            first = 0
            last = period
            x = np.array(list(range(first,last))).reshape(-1, 1) 
            y= np.array(close).reshape(-1, 1) 
            from sklearn.linear_model import LinearRegression
            reg = LinearRegression().fit(x,y)

            max_high1 = 0
            max_high2 = 0
            max_low1 = 0
            max_low2 = 0
            for index in range(len(close)):
                predicted_price = reg.predict(np.array([[index]]))[0][0]
                high_diff = high[index] - predicted_price
                if(max_high1<high_diff):
                    max_high2 = max_high1
                    max_high1 = high_diff
                else:
                    if(max_high2<high_diff):
                        max_high2 = high_diff

                low_diff =  predicted_price - low[index]
                if(max_low1<low_diff):
                    max_low2 = max_low1
                    max_low1 = low_diff
                else:
                    if(max_low2<low_diff):
                        max_low2 = low_diff
            predicted_close = reg.predict(np.array([[len(close)+skip]]))[0][0]
            linreg_upper[i] = predicted_close+((max_high1+max_high2)/2.0)
            linreg_lower[i] = predicted_close-((max_low1+max_low2)/2.0 )
    return linreg_upper, linreg_lower

def PAST_LOW_HIGH(data, period = 14):
    slop_angle = np.full(len(data), np.nan)
    past_high = np.full(len(data), np.nan)
    past_low = np.full(len(data), np.nan)
    for i in range(len(data)):
        if(i>=period):
            slop_angle[i] = SLOPE(data[i-period:i].close)
            past_high[i] = data[0:i-1].tail(period).high.max()
            past_low[i] = data[0:i-1].tail(period).low.min()
    return slop_angle, past_high, past_low
    
def process_data(quotes):
    from talib import RSI
    from talib import MACD
    from talib import SMA
    from talib import EMA
    from talib import ADX
    from talib import BBANDS
    from talib import STOCHRSI
    from talib import LINEARREG_ANGLE
    quotes['stoploss'] = np.full(len(quotes), np.nan)
    # Simple Moving Averages
    # quotes['sma5'] = SMA(quotes['close'], timeperiod=2)
    # quotes['sma20'] = SMA(quotes['close'], timeperiod=20)
    # quotes['sma50'] = SMA(quotes['close'], timeperiod=50)
    # quotes['sma100'] = SMA(quotes['close'], timeperiod=100)
    #quotes['sma20'] = SMA(quotes['close'], timeperiod=20)
    
    # Exponential Moving Averages
    quotes['ema50'] = EMA(quotes['close'], timeperiod=50)
    quotes['ema100'] = EMA(quotes['close'], timeperiod=100)
    quotes['ema200'] = EMA(quotes['close'], timeperiod=200)
    quotes['ema20'] = EMA(quotes['close'], timeperiod=20)
    quotes['ema12'] = EMA(quotes['close'], timeperiod=12)
    quotes['ema7'] = EMA(quotes['close'], timeperiod=7)
    quotes['ema5'] = EMA(quotes['close'], timeperiod=5)
    quotes['regression_ema7'] =  LINEARREG_ANGLE(quotes["ema7"], timeperiod=3)
    quotes['regression_ema50'] =  LINEARREG_ANGLE(quotes["ema50"], timeperiod=3)
    
    # MACD indicator - MACD(12,26,9)
    quotes['macd_line'], quotes['signal_line'], quotes['macd_hist'] = MACD(quotes['close'], fastperiod=12, slowperiod=26, signalperiod=9)

    quotes["rsi_14"] = RSI(quotes['close'].values, timeperiod=14)
    #quotes["stoch_rsi_fastk_10"],quotes["stoch_rsi_fastd_8"] = STOCHRSI(quotes['close'].values, timeperiod=14, fastk_period=10, fastd_period=8, fastd_matype=0)
    
    
    #quotes["atr_stoploss_3_14"] = ATR_STOPLOSS(close = quotes.close.values, high = quotes.high.values, low = quotes.low.values, 
    #                             times=3, period=14)[0]
    #quotes["atr_stoploss_2_10"] = ATR_STOPLOSS(close = quotes.close.values, high = quotes.high.values, low = quotes.low.values, 
    #                             times=2, period=10)[0]
    
    quotes["adx_14"] = ADX(quotes['high'].values, quotes['low'].values, quotes['close'].values, timeperiod=14)

    #quotes["bb_upperband_5"],quotes["bb_middleband_5"],quotes["bb_lowerband_5"] = BBANDS(quotes['close'].values, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    #quotes["slope_angle_60"],quotes["past_high_60"],quotes["past_low_60"] = PAST_LOW_HIGH(quotes, period = 60)
    #quotes["slope_angle_180"],quotes["past_high_180"],quotes["past_low_180"] = PAST_LOW_HIGH(quotes, period = 180)
    #quotes["slope_angle_30"],quotes["past_high_30"],quotes["past_low_30"] = PAST_LOW_HIGH(quotes, period = 30)
    #quotes["slope_angle_12"],quotes["past_high_12"],quotes["past_low_12"] = PAST_LOW_HIGH(quotes, period = 12)
    #quotes["slope_angle_7"],quotes["past_high_7"],quotes["past_low_7"] = PAST_LOW_HIGH(quotes, period = 7)
    
    
    from talib import ATR
    quotes["atr_risk"]= ATR(quotes['high'].values, quotes['low'].values, quotes['close'].values ,timeperiod=14)
    
    
    keltner_multiplier = 2
    quotes["atr_10"]= ATR(quotes['high'].values, quotes['low'].values, quotes['close'].values ,timeperiod=10)
    quotes["keltner_upper"] = quotes['ema20'] + (keltner_multiplier*quotes["atr_10"])
    quotes["keltner_lower"] = quotes['ema20'] - (keltner_multiplier*quotes["atr_10"])
    '''
    quotes["linear_reg_upper_200"], quotes["linear_reg_lower_200"] = LINEAR_REGRESSION_BOUND(np.array(quotes["close"]),
                                                                                     np.array(quotes["high"]),
                                                                                     np.array(quotes["low"]),200,0)
    quotes["linear_reg_upper_50"], quotes["linear_reg_lower_50"] = LINEAR_REGRESSION_BOUND(np.array(quotes["close"]),
                                                                                     np.array(quotes["high"]),
                                                                                     np.array(quotes["low"]),50,0)
    quotes["linear_reg_upper_30"], quotes["linear_reg_lower_30"] = LINEAR_REGRESSION_BOUND(np.array(quotes["close"]),
                                                                                     np.array(quotes["high"]),
                                                                                     np.array(quotes["low"]),30,0)
    quotes["linear_reg_upper_30_5"], quotes["linear_reg_lower_30_5"] = LINEAR_REGRESSION_BOUND(np.array(quotes["close"]),
                                                                                     np.array(quotes["high"]),
                                                                                     np.array(quotes["low"]),30,5)
    
    '''
    return quotes


# 1. Strategy



In [10]:
class StrategyTurtle(Strategy):
    def get_action(self, data, i, shares_can_buy, shares_can_sell):
        import random
        action ="HOLD", 0, data
        
        enter_fast = 20
        exit_fast = 10
        enter_slow = 55
        exit_slow = 20
        
        fastL = data["close"][i-enter_fast:i].max()
        fastLC = data["close"][i-exit_fast:i].min()
        fastS = data["close"][i-enter_fast:i].min()
        fastSC =data["close"][i-exit_fast:i].max()

        slowL = data["close"][i-enter_slow:i].max()
        slowLC = data["close"][i-exit_slow:i].min()
        slowS = data["close"][i-enter_slow:i].min()
        slowSC = data["close"][i-exit_slow:i].max()

        enterL1 = data["high"][i] > fastL
        exitL1 = data["low"][i] <= fastLC
        enterS1 = data["low"][i] < fastS
        exitS1 = data["high"][i] >= fastSC

        enterL2 = data["high"][i] > slowL
        exitL2 = data["low"][i] <= slowLC
        enterS2 = data["low"][i] < slowS
        exitS2 = data["high"][i] >= slowSC
        
        if(enterL1 or enterL2):
            data = self.set_stoploss(data, i , 14, 3)
            action = "BUY", data.close[i], data
        if(exitL1 or exitL2):
            action = "SELL", data.close[i], data
            
        
        if(shares_can_sell>0):
            if(i>0 
               and data["stoploss"][i]>data.low[i]
              ):
                action = "STOPPED", data["stoploss"][i], data
        
        return action
    
    

# 2. Back Testing

In [286]:
def multi_backtest(stocks, total_tries, strategy=[StrategyRandom()], cash = 10000, transaction_fee =0.01):
    accumulated_growth = 0
    tries = 0
    for i in range(0,total_tries):
        for stock in stocks:
            processed_data=process_data(get_data(stock))
            for strgy in strategy:
                print(stock+" # "+strgy.__class__.__name__)
                result = back_test(processed_data, cash = cash, transaction_fee =transaction_fee, 
                                   strategy=strgy)
                accumulated_growth = accumulated_growth + result[0]
                if(result[0]!=0):
                    tries = tries+ 1
    print("########## "+str(round(accumulated_growth/tries,2))+"% ######## tries: "+str(tries))
                    
#strategies = [StrategyRandom(),StrategyTurtle(),StrategyTrend()]
strategies = [StrategyTrend()]

stocks  = get_stocks()[250:300]["stock"].values
multi_backtest(stocks, 1, strategy=strategies, cash = 100000, transaction_fee =0.01)

WDAY # StrategyTrend
asset: 101698.09846367317
profit: 1.7%
CRARY # StrategyTrend
asset: 100000
profit: 0.0%
VLO # StrategyTrend
asset: 100000
profit: 0.0%
HKXCY # StrategyTrend
asset: 99460.3283199845
profit: -0.54%
HKXCF # StrategyTrend
asset: 99310.49185228362
profit: -0.69%
MFG # StrategyTrend
asset: 100000
profit: 0.0%
REGN # StrategyTrend
asset: 100000
profit: 0.0%
LRCX # StrategyTrend
asset: 103319.97284052949
profit: 3.32%
SCHYY # StrategyTrend
asset: 100505.87275289267
profit: 0.51%
CM # StrategyTrend
asset: 100000
profit: 0.0%
SCHYF # StrategyTrend
asset: 100000
profit: 0.0%
TEFOF # StrategyTrend
asset: 100000
profit: 0.0%
TEF # StrategyTrend
asset: 100000
profit: 0.0%
MFC # StrategyTrend
asset: 100000
profit: 0.0%
ETN # StrategyTrend
asset: 98412.0628595616
profit: -1.59%
PRU # StrategyTrend
asset: 100000
profit: 0.0%
BBT-PH # StrategyTrend
asset: 100000
profit: 0.0%
BBT-PG # StrategyTrend
asset: 98601.43493213816
profit: -1.4%
KHC # StrategyTrend
asset: 100000
profit: 0.0%


# 3. Chart for analysis

In [11]:

def plot_chart(processed_df, strategy):
    result = back_test(processed_df, cash = 100000, transaction_fee =0.01, assets={}, strategy=strategy)
    ######Plot
    ######
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    # Make sure dates are in ascending order
    # We need this for slicing in the callback below
    df = processed_df.set_index('date')

    fig = go.FigureWidget(make_subplots(rows=4, cols=1))
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['close']),
        name = "close"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema20']),
        name = "ema20"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema7']),
        name = "ema7"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema5']),
        name = "ema5"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema12']),
        name = "ema12"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema50']),
        name = "ema50"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema100']),
        name = "ema100"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['ema200']),
        name = "ema200"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['keltner_upper']),line = dict(color='black', width=1, dash='dash'),
        name = "keltner_upper"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['keltner_lower']),line = dict(color='black', width=1, dash='dash'),
        name = "keltner_lower"),
        row=1, col=1
    )
    
    '''
    
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['linear_reg_upper_200']),
        name = "linear_reg_upper_200"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['linear_reg_lower_200']),
        name = "linear_reg_lower_200"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['linear_reg_upper_50']),
        name = "linear_reg_upper_50"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['linear_reg_lower_50']),
        name = "linear_reg_lower_50"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['linear_reg_upper_30']),
        name = "linear_reg_upper_30"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['linear_reg_lower_30']),
        name = "linear_reg_lower_30"),
        row=1, col=1
    )
    '''
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['stoploss']), line = dict(color='firebrick', width=2, dash='dash'),
        name = "stoploss"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=[i for i in result[2]], y=[ result[2][i] for i in result[2]],
        name = "Short", mode="markers", marker=dict(color="red", size=5)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=[i for i in result[1]], y=[df.loc[i]["close"] for i in result[1]],
        name = "Long", mode="markers", marker=dict(color="green", size=5)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['rsi_14']),
        name = "rsi_14"),
        row=2, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['adx_14']),
        name = "adx_14"),
        row=3, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['macd_line']),
        name = "macd_line"),
        row=4, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['signal_line']),
        name = "signal_line"),
        row=4, col=1
    )
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig.layout.yaxis=dict(
            domain=[0.53, 1]
        )
    fig.layout.yaxis2=dict(
            domain=[0.35, 0.5]
        )
    fig.layout.yaxis3=dict(
            domain=[0.18, 0.33]
        )
    fig.layout.yaxis4=dict(
            domain=[0, 0.16]
        )
    fig.layout.yaxis2.range=[0,100]
    fig.layout.yaxis2.tickvals=[0,20,80,100]
    fig['layout'].update(height=600, width=1000) 
    
        
    #trace = go.Candlestick(x=df.index,open=df['open'],high=df['high'],low=df['low'],close=df['close'])
    
    import time
    import datetime
    def zoom(layout, xrange):
        try:
            fig.layout.xaxis2.range = fig.layout.xaxis.range
            fig.layout.xaxis3.range = fig.layout.xaxis.range
            fig.layout.xaxis4.range = fig.layout.xaxis.range
            in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
            padding = 0.1
            
            from_price = in_view.iloc[0].close
            to_price = in_view.iloc[-1].close
            growth = (to_price-from_price)/from_price*100.0
            
            max_val_y = in_view.close.max()
            min_val_y =  in_view.close.min()
            axis_diff = max_val_y-min_val_y
            fig.layout.yaxis.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
            
            max_val_y = in_view["adx_14"].max()
            min_val_y =  in_view["adx_14"].min()
            axis_diff = max_val_y-min_val_y
            fig.layout.yaxis3.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            max_val_y = in_view["macd_line"].max()
            min_val_y =  in_view["macd_line"].min()
            axis_diff = max_val_y-min_val_y
            fig.layout.yaxis4.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            fig['layout'].update(title=symbol+" ("+str(round(growth,2))+"%)") 
        except Exception as e:
            None
    fig.layout.on_change(zoom, 'xaxis')
    return fig

75.96375653207353

In [280]:

class StrategyTrend(Strategy):
    def get_action(self, data, i, shares_can_buy, shares_can_sell):
        import random
        action ="HOLD", 0, data
        if(shares_can_buy >0):
            if(i>210):
                if(
                    (
                        ######### filter up trend #########
                        data["ema200"][i]<data["ema20"][i]
                        and data["regression_ema50"][i]>0
                        ######### Long term trend ###########
                    )
                    and
                    (
                        ####### filter up trend #######
                        data["regression_ema7"][i]>0
                        and SLOPE(data["high"][i-2:i+1])>0
                        and SLOPE(data["low"][i-2:i+1])>0
                        and data["low"][i] >data["low"][i-1]
                        and data["high"][i] >data["high"][i-1]
                        ######### Short term trend ######
                    )
                    and
                    (
                        (
                            ####### ema7 cross ema20 #######
                            data["ema20"][i]<data["ema7"][i]
                            and data["ema20"][i-1]>data["ema7"][i-1]
                            ######### BUY SIGNAL ###########
                        )
                        or False
                    )
                    and 
                    (
                    ######### BUY SIGNAL ##################
                       (
                            ####### ema7 cross ema20 #######
                            data["ema20"][i]<data["ema7"][i]
                            and data["ema20"][i-1]>data["ema7"][i-1]
                            ####### MACD cross over in the last 5 days #######
                            and data["macd_hist"][i-5:i].min()<0
                        )
                        or 
                        (
                            ####### macd cross over #######
                            data["macd_hist"][i-1]<0
                            and data["macd_hist"][i]>0
                        )
                    )
                    and
                    (
                    ################# confirmation signal #################    
                        (
                            ####### MOMENTUM Strong ADX going up #######
                            SLOPE(data["adx_14"][i-3:i].values)>SLOPE(data["adx_14"][i-7:i].values)
                            and SLOPE(data["adx_14"][i-3:i])>0
                        )
                        or
                        (
                            ####### Strong Trend #######
                            SLOPE(data["ema100"][i-10:i]-data["ema200"][i-10:i])>1
                        )
                    )
                ):
                    data = self.set_stoploss(data, i , 7, 3)
                    action="BUY", data["close"][i], data
       
        
        if(shares_can_sell>0):
            if(i>210
                    and data["stoploss"][i]>data.close[i]
                  ):
                    action = "STOPPED", data["stoploss"][i], data
        
        if(shares_can_sell>0):
            if(False
              ):
                action = "SELL", data["close"][i], data
        
        return action

stock="GOOGL"
stock="NFLX"
stock="BAC-PY"
stock="WFC-PT"
stock="WFC-PN"
stock="AAPL"
stock="D05.SI"
stock="TSLA"

sdate='2015-01-01'
edate='2019-12-01'
data = pd.read_sql_query("SELECT * FROM finance.stock_price where stock = '"+stock+"' and date between '"+sdate+"' and '"+edate+"' order by date asc",con=connection)
processed_df = process_data(data)
plot_chart(processed_df, StrategyTrend())

asset: 102894.74823668812
profit: 2.89%


FigureWidget({
    'data': [{'name': 'close',
              'type': 'scattergl',
              'uid': '35eb40a…

In [13]:
            
    ##filter up trend stock
    if(i>210):
                if(
                    data["ema20"][i]<data["ema7"][i]
                    and data["macd_line"][i]>0
                    #and (data["ema5"][i-50:i]-data["ema100"][i-50:i]).min()>0
                    and SLOPE(data["ema7"][i-3:i]-data["ema20"][i-3:i])>0
                    and SLOPE(data["ema7"][i-3:i]-data["ema20"][i-3:i])>0
                    and data["regression_ema7"][i]>=0.75
                ):
                    data = self.set_stoploss(data, i , 7, 3)
                    action="BUY", data["close"][i], data
       

NameError: name 'i' is not defined

In [ ]:
        if(i>100
               and 
               (
                   SLOPE(data["close"][i-2:i])>0
                   and 
                    (
                        SLOPE(data["close"][i-50:i])>0.1
                        or
                        (data["rsi_14"][i]>20
                            and data["rsi_14"][i-3:i].min()<20
                            and data["rsi_14"][i-3:i].max()<data["rsi_14"][i])
                        or
                        (data["macd_hist"][i]>0
                            and data["macd_hist"][i-5:i].min()<0
                            and data["macd_hist"][i]>data["macd_hist"][i-1]
                            and SLOPE(data["macd_hist"][i-2:i])>0
                        )
                    )
                    and(
                    
                        (
                             data["ema7"][i]>data["ema20"][i]
                            and data["ema7"][i-1]<data["ema20"][i-1]
                            and SLOPE(data["adx_14"][i-2:i])>0
                        )
                            or
                        (
                             data["ema20"][i]>data["ema50"][i]
                            and data["ema20"][i-1]<data["ema50"][i-1]
                            and SLOPE(data["adx_14"][i-2:i])>0
                       )
                        or
                        (
                            data["ema7"][i]>data["ema20"][i]
                            and data["ema20"][i]>data["ema50"][i]
                            and (
                                (data["ema7"][i-3:i]-data["ema20"][i-3:i]).min()<0
                                or
                                (data["ema20"][i-7:i]-data["ema50"][i-7:i]).min()<0
                            )
                            and SLOPE(data["adx_14"][i-2:i])>0
                            and SLOPE(data["ema7"][i-2:i] - data["ema20"][i-2:i])>0
                            and SLOPE(data["ema20"][i-2:i] - data["ema50"][i-2:i])>0
                       )
                    )
               )
            ):

In [ ]:
            
    #14% AAPL
    if(i>100
               and 
               (
                    (
                        SLOPE(data["close"][i-50:i])>0.1
                        or
                        (data["rsi_14"][i]>20
                            and data["rsi_14"][i-3:i].min()<20)
                        or
                        (data["macd_hist"][i]>0
                            and data["macd_hist"][i-5:i].min()<0
                            and data["macd_hist"][i]>data["macd_hist"][i-1]
                            and SLOPE(data["macd_hist"][i-3:i])>0
                        )
                    )
                    and(
                    
                        (
                             data["ema7"][i]>data["ema20"][i]
                            and data["ema7"][i-1]<data["ema20"][i-1]
                        )
                            or
                        (
                             data["ema20"][i]>data["ema50"][i]
                            and data["ema20"][i-1]<data["ema50"][i-1]
                       )
                    )
                    and (SLOPE(data["adx_14"][i-2:i])>0
                            and 
                         SLOPE(data["adx_14"][i-4:i-1])<SLOPE(data["adx_14"][i-2:i])
                        )
               )
            ):

In [ ]:
    #21% AAPL       
    if(i>100
               and 
               (
                    (
                        SLOPE(data["close"][i-50:i])>0.1
                        or
                        (data["rsi_14"][i]>20
                            and data["rsi_14"][i-3:i].min()<20)
                        or
                        (data["macd_hist"][i]>0
                            and data["macd_hist"][i-5:i].min()<0
                            and data["macd_hist"][i]>data["macd_hist"][i-1]
                        )
                    )
                    and(
                    
                        (
                             data["ema7"][i]>data["ema20"][i]
                            and data["ema7"][i-1]<data["ema20"][i-1]
                        )
                            or
                        (
                             data["ema20"][i]>data["ema50"][i]
                            and data["ema20"][i-1]<data["ema50"][i-1]
                       )
                    )
                   and SLOPE(data["adx_14"][i-3:i+1])>0
               )
            ):